In [2]:
import pandas as pd
import numpy as np
import requests

from pandas.io import gbq
from google.cloud import bigquery,storage
from google.oauth2 import service_account

import gspread

from canvasapi import Canvas

import plotly.express as px

from instances import env_keys
#API_KEY = env_keys['API_key']
API_KEY = env_keys['ACCES_TOKEN']
API_URL = env_keys['API_URL']#+'/accounts/1'

In [3]:
project_id = 'canvas-portal-data-custom'
cred_file = 'canvas-portal-data-custom-6e244db3b826.json'
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)

In [4]:
# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)
canvas.__dict__

{'_Canvas__requester': <canvasapi.requester.Requester at 0x1141fc438>}

### Get courses running on Canvas -- created via migration or by LT's

In [40]:
#Get the LT list from the Excel sheet
lt_df_cols = ['School', 'Dept_num', 'Dept_name', 'Name', 'Email']
lt_df = pd.read_excel('Learning_Technologists_updating.xlsx')
lt_df.columns = lt_df_cols
lt_df['Email'] = lt_df.Email.str.lower()
lt_df.tail()

,School,Dept_num,Dept_name,Name,Email
39,?,ESG,ESG,Mariela M Perez-Cabarcas,mmpc2000@mit.edu
40,?,HST,HST,Keegan Mendez,kmendez@mit.edu
41,A,MAS,Media Lab,Kaili Glasser,glasserk@mit.edu
42,?,OL,Open Leearning,Lorcan Murphy,murphyl@mit.edu
43,H,WGS,Women's and Gender Studies,Anna Williams,ajlwill@mit.edu


In [11]:
#Read the Stellar to Canvas migration list on Google Drive, and construct a
#list of course_id s
gs_name = "Stellar to Canvas content migration request (Responses)"
#gs_name = "xyz"
gc = gspread.service_account(filename=cred_file)
sh = gc.open(gs_name).sheet1
sh_data = sh.get_all_values()
head_col = sh_data.pop(0)
stellar_df = pd.DataFrame(sh_data, columns=head_col)
stellar_df['course_id'] = stellar_df['Canvas URL to migrate to'].str.split("/").str[-1]

In [13]:
def get_course_info(course_id):
    '''This function gets the course information and the file/assignment update times,
    by the course_id, and returns a list of '''
    try:
        c1 = canvas.get_course(course_id)


        course_dept = sub_account_dict[c1.account_id]
        course_name = c1.name

        files_ = c1.get_files()
        assn_ = c1.get_assignments()
        #Get the file updated times
        file_utimes = [f_.updated_at for f_ in files_]

        #Get the assignment updated times
        assn_utimes = [a_.updated_at for a_ in assn_]
        fa_times = file_utimes + assn_utimes

        #Convert the whole thing 
        fa_times = np.array(fa_times, dtype='datetime64')



        return [c1.id, course_dept, course_name, len(file_utimes), len(assn_utimes), 
                len(fa_times), fa_times.max()]
    except Exception as e:
        return [None, None, None, None, None, None, None]

In [45]:
#Get a list of all department names by the sub-account id:
acc = canvas.get_account(1)
sub_account_dict = {}
accs = acc.get_subaccounts(recursive=True)
for a_ in accs:
    sub_account_dict[a_.id] = a_.name

#sub_account_dict

In [46]:
#Change line #7 for full list
stellar_df_course_list = stellar_df.course_id.tolist()
migrated_course_rows = []
migrated_course_cols = ['course_id','Dept', 'Course_name', 'num_files', 'num_assignments', 'num_tot_fa',
                        'last_update_at']

for c_id in stellar_df_course_list[:3]:
    migrated_course_rows.append(get_course_info(c_id))
    
migrated_course_df = pd.DataFrame.from_records(migrated_course_rows, columns=migrated_course_cols)
migrated_course_df['if_LT_led'] = 0
migrated_course_df['LT_email'] = np.nan
migrated_course_df.tail()


/Users/anindyaroy/Sites/fun/venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



,course_id,Dept,Course_name,num_files,num_assignments,num_tot_fa,last_update_at,if_LT_led,LT_email
0,4631,7 - Department of Biology,7.012 Introductory Biology,59,34,93,2020-06-30 22:32:48,0,NaN
1,4579,7 - Department of Biology,7.493 Microbial Genetics and Evolution,105,45,150,2020-06-26 21:40:23,0,NaN
2,4691,CMS - Program in Comparative Media Studies,CMS.594 Education Technology Studio,40,34,74,2020-06-30 04:53:54,0,NaN


In [47]:
#Change line #5 for full list
lt_courses_row = []
lt_courses_cols = migrated_course_cols + ['if_LT_led','LT_email']
courses_to_exclude = [3157, 3158]
for user_email in lt_df.Email.tolist()[:3]:
    try:
        user_ = canvas.get_user(user_email, 'sis_login_id')
        course_list = []
        user_courses = user_.get_enrollments(type=['TeacherEnrollment'])

        for uc_ in user_courses:
            #print(uc_.id, uc_.course_id)
            if uc_.course_id not in courses_to_exclude:
                uc_row = get_course_info(uc_.course_id)
                #print(uc_row)
                uc_row.extend([1, user_email])
                lt_courses_row.append(uc_row)
    except Exception as e:
        print('Error {} for user {}'.format(e, user_email))
        pass

        
all_lt_courses = pd.DataFrame.from_records(lt_courses_row, columns=lt_courses_cols)
all_lt_courses


/Users/anindyaroy/Sites/fun/venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



,course_id,Dept,Course_name,num_files,num_assignments,num_tot_fa,last_update_at,if_LT_led,LT_email
0,5051,Sandboxes,Temporary Sandbox,30,7,37,2020-06-24 20:11:37,1,zroberts@mit.edu
1,3170,1 - Department of Civil and Environmental Engi...,1.008 Eng Solutions to Soc Change,84,4,88,2020-06-12 17:34:00,1,zroberts@mit.edu
2,3165,1 - Department of Civil and Environmental Engi...,1.000 Intro to Comp Programming,22,4,26,2020-05-28 02:22:05,1,zroberts@mit.edu
3,3305,1 - Department of Civil and Environmental Engi...,1.723 Computational Methods,22,4,26,2020-05-28 02:22:34,1,zroberts@mit.edu
4,5397,1 - Department of Civil and Environmental Engi...,"1.74 Land, Water, Food, and Climate",99,5,104,2020-06-30 15:07:24,1,zroberts@mit.edu
5,5005,Sandboxes,Temporary Sandbox,23,4,27,2020-06-03 18:30:11,1,hcelio@mit.edu
6,5068,Sandboxes,Demo Course,27,2,29,2020-06-19 21:49:29,1,mrbraun@mit.edu


In [42]:
all_courses_df = pd.concat([migrated_course_df, all_lt_courses], ignore_index=True)

In [43]:
ts_cutoff_1w = pd.to_datetime('now') - pd.to_timedelta('7days')
all_courses_df['if_active_last_week'] = np.where(all_courses_df.last_update_at>ts_cutoff_1w, 1, 0)
all_courses_df['if_sandbox_course'] = np.where(all_courses_df.Dept=='Sandboxes', 1, 0)
all_courses_df = all_courses_df[all_courses_df.course_id.notna()].reset_index(drop=True)
all_courses_df.drop_duplicates(subset=['course_id'], keep='last', inplace=True)
all_courses_df.tail()

,course_id,Dept,Course_name,num_files,num_assignments,num_tot_fa,last_update_at,if_LT_led,LT_email,if_active_last_week,if_sandbox_course
219,5050.0,Sandboxes,Countdown to OQE 2020,24.0,8.0,32.0,2020-06-22 14:08:26,1,kmendez@mit.edu,0,1
220,4724.0,HST - Program in Health Sciences and Technology,Evolutionary & Quant Genomics,19.0,51.0,70.0,2020-06-23 11:32:04,1,kmendez@mit.edu,0,0
221,4999.0,Sandboxes,Temporary Sandbox,22.0,5.0,27.0,2020-06-03 18:49:18,1,glasserk@mit.edu,0,1
222,5074.0,Sandboxes,Temporary Sandbox,30.0,5.0,35.0,2020-07-01 21:48:21,1,murphyl@mit.edu,1,1
223,4983.0,Sandboxes,Temporary Sandbox,24.0,13.0,37.0,2020-06-12 19:28:22,1,ajlwill@mit.edu,0,1


In [44]:
all_courses_df['last_update_at'] = all_courses_df['last_update_at'].dt.strftime('%Y-%m-%d')
all_courses_df.tail()

,course_id,Dept,Course_name,num_files,num_assignments,num_tot_fa,last_update_at,if_LT_led,LT_email,if_active_last_week,if_sandbox_course
219,5050.0,Sandboxes,Countdown to OQE 2020,24.0,8.0,32.0,2020-06-22,1,kmendez@mit.edu,0,1
220,4724.0,HST - Program in Health Sciences and Technology,Evolutionary & Quant Genomics,19.0,51.0,70.0,2020-06-23,1,kmendez@mit.edu,0,0
221,4999.0,Sandboxes,Temporary Sandbox,22.0,5.0,27.0,2020-06-03,1,glasserk@mit.edu,0,1
222,5074.0,Sandboxes,Temporary Sandbox,30.0,5.0,35.0,2020-07-01,1,murphyl@mit.edu,1,1
223,4983.0,Sandboxes,Temporary Sandbox,24.0,13.0,37.0,2020-06-12,1,ajlwill@mit.edu,0,1


In [27]:
all_courses_df[all_courses_df.if_sandbox_course==0]
all_courses_df.to_gbq('lt_courses.all_courses', project_id, if_exists='replace', credentials=credentials)
#all_courses_df.to_csv('all_courses.csv', index=None)

1it [00:04,  4.14s/it]
